In [1]:
import pandas as pd
import numpy as np

In [10]:
# Get datasets to be merged
url = 'https://github.com/BrownRegaSterlingHeinen/PostsecondaryAttainment/raw/master/2016/NCCC%20Datasets/mergedData.xlsx'
mergedNCData = pd.read_excel(url)

url2 = 'https://github.com/jakemdrew/EducationDataNC/raw/master/2016/School%20Datasets/PublicHighSchools2016.csv'
hsData = pd.read_csv(url2)

In [3]:
# View info
mergedNCData.info()
hsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Columns: 721 entries, County to $200,000 or more Number of Returns
dtypes: float64(365), int64(68), object(288)
memory usage: 332.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Columns: 1256 entries, vphone_ad to WhitePct
dtypes: float64(1215), int64(4), object(37)
memory usage: 4.6+ MB


In [4]:
# Create function to get weighted average
# http://pbpython.com/weighted-average.html
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [12]:
# One-hot encode important categorical variables
hsData = pd.get_dummies(data=hsData, columns=["SPG Grade", "EVAAS Growth Status"])

In [79]:
newhsData = pd.DataFrame()
columns = list(hsData)
for col in columns:
    try:
        newhsData[col] = hsData.groupby("Lea_Name").apply(wavg, col, "student_num")
    except:
        print("Column", col, "could not be weighted")

Column vphone_ad could not be weighted
Column street_ad could not be weighted
Column scity_ad could not be weighted
Column state_ad could not be weighted
Column type_cd could not be weighted
Column category_cd could not be weighted
Column url_ad could not be weighted
Column grade_range_cd could not be weighted
Column calendar_type_txt could not be weighted
Column sna_pgm_type_cd could not be weighted
Column cover_letter_ad could not be weighted
Column school_type_txt could not be weighted
Column calendar_only_txt could not be weighted
Column title1_type_cd could not be weighted
Column clp_ind could not be weighted
Column focus_clp_ind could not be weighted
Column summer_program_ind could not be weighted
Column Lea_Name could not be weighted
Column School_Name could not be weighted
Column State_Name could not be weighted
Column esea_status could not be weighted
Column Grad_project_status could not be weighted
Column stem could not be weighted
Column url could not be weighted
Column Dist

In [85]:
# Get district names to bring back to DF
keys = hsData.groupby("Lea_Name")
keys = keys.groups.keys()
keys = list(keys)

In [86]:
newhsData["Lea_Name"] = keys

In [89]:
newhsData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, Alamance-Burlington Schools to Yancey County Schools
Columns: 1229 entries, year to Lea_Name
dtypes: float64(1228), object(1)
memory usage: 1.1+ MB


In [24]:
# If you wanted to use the max student number instead of weighed average
# https://stackoverflow.com/questions/15705630/python-getting-the-row-which-has-the-max-value-in-groups-using-groupby
# hsDataMax = hsData.groupby(['Lea_Name'])['student_num'].transform(max) == hsData['student_num']
# newhsData = hsData[hsDataMax]

In [90]:
# Create county name for merging
newhsData['County'] = newhsData['Lea_Name'].str.replace('Schools', '')
newhsData['County'] = newhsData['County'].str.replace('County', '')
newhsData['County'] = newhsData['County'].str.replace('Public', '')
newhsData['County'] = newhsData['County'].str.rstrip()

In [91]:
# Deal with mis-matched county names
newhsData['County'] = newhsData['County'].str.replace('Alamance-Burlington', 'Alamance')
newhsData['County'] = newhsData['County'].str.replace('Charlotte-Mecklenburg', 'Mecklenburg')
newhsData['County'] = newhsData['County'].str.replace('Edgecombe   School', 'Edgecombe')
newhsData['County'] = newhsData['County'].str.replace('Cumberland', 'Fayetteville')
newhsData['County'] = newhsData['County'].str.replace('Nash-Rocky Mount', 'Nash')
newhsData['County'] = newhsData['County'].str.replace('Rowan-Salisbury', 'Rowan')
newhsData['County'] = newhsData['County'].str.replace('Edenton-Chowan', 'Chowan')

In [94]:
mergedWithHS = pd.merge(mergedNCData, newhsData, on = "County")

In [95]:
mergedWithHS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Columns: 1950 entries, County to Lea_Name
dtypes: float64(1593), int64(68), object(289)
memory usage: 899.3+ KB


In [97]:
mergedWithHS.to_excel('mergedData_withHS.xlsx', index=False)